In [1]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import pandas as pd
from cloudy_fit_lib import *
from scipy.interpolate import RegularGridInterpolator
import pickle

In [2]:
plt.style.use('/Users/thepoetoftwilight/Documents/Astro/Code/PG1522+101/science.mplstyle')

In [3]:
rootdir = '/Users/thepoetoftwilight/Documents/Astro/Data/PG1522+101/cloudy_grids/'

Obtain the UVB grid from FG20

In [4]:
fg20_z_grid, fg20_wav_grid, fg20_J_nu_grid = read_uvb('', 'fg20_galaxy.ascii')

Fetch the SED at $z=1$

In [5]:
z_test = 1

In [6]:
fg20_J_nu_test = fetch_sed(z_test, fg20_z_grid, fg20_J_nu_grid)

Specify grid points for densities and metallicities

In [8]:
log_hdens_min_main = -4.5
log_hdens_max_main = -2.5
log_hdens_step_main = 0.25

log_hdens_arr_main = np.arange(log_hdens_min_main, log_hdens_max_main+log_hdens_step_main, log_hdens_step_main)

In [9]:
log_metals_min_main = -0.5
log_metals_max_main = 1.0
log_metals_step_main = 0.25

log_metals_arr_main = np.arange(log_metals_min_main, log_metals_max_main+log_metals_step_main, log_metals_step_main)

In [13]:
log_metals_arr_main

array([-0.5 , -0.25,  0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [15]:
# Total number of grid points
len(log_hdens_arr_main)*len(log_metals_arr_main)

63

Check compile with $N_\mathrm{HI}$

In [17]:
# Get list of densities and metallicities for this stopping HI column density 
log_hdens_grid, log_metals_grid = read_grd_file(rootdir+'PIE_fixT/', 'igm_lalpha_hm12_grid_1300')

# Get column densities for all species
species_names, log_col_dens_grid = read_col_file(rootdir+'PIE_fixT/', 'igm_lalpha_hm12_grid_1300')

for j in range(len(log_hdens_arr_main)):

    log_hdens = log_hdens_arr_main[j]

    for k in range(len(log_metals_arr_main)):

        log_metals = log_metals_arr_main[k]

        # Get grid index number for the current n_H and metallicity
        idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]

        # Isolate column density for all species
        log_col_dens = log_col_dens_grid[idx]

        # Check if this grid point is a failure
        if np.round(log_col_dens[0],2) != 12.9:
            # Categorize based on temperature
            print(log_hdens, log_metals, log_col_dens[0])

/Users/thepoetoftwilight/Documents/Astro/Code/PG1522+101/cloudy_fit/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))


Compile 2D grid

In [18]:
species_logN_samples_final = {}

In [19]:
# Get list of densities and metallicities for this stopping HI column density 
log_hdens_grid, log_metals_grid = read_grd_file(rootdir+'PIE_fixT/', 'igm_lalpha_hm12_grid_1300')

# Get column densities for all species
species_names, log_col_dens_grid = read_col_file(rootdir+'PIE_fixT/', 'igm_lalpha_hm12_grid_1300')

for j in range(len(log_hdens_arr_main)):

    log_hdens = log_hdens_arr_main[j]

    for k in range(len(log_metals_arr_main)):

        log_metals = log_metals_arr_main[k]

        # Get grid index number for the current n_H and metallicity
        idx = np.intersect1d(np.where(log_hdens_grid==log_hdens)[0], np.where(log_metals_grid==log_metals)[0])[0]
        # Isolate column density for all species
        log_col_dens = log_col_dens_grid[idx]

        # For each species
        for l in range(len(species_names)):

            s = species_names[l]

            if s not in species_logN_samples_final.keys():

                species_logN_samples_final[s] = -99.*np.ones((len(log_hdens_arr_main),
                                                        len(log_metals_arr_main)))

            # Check for converged logN(HI)
            if np.round(log_col_dens[0],2) == 12.9:
                species_logN_samples_final[s][j,k] = log_col_dens[l]

/Users/thepoetoftwilight/Documents/Astro/Code/PG1522+101/cloudy_fit/cloudy_fit_lib.py:434: RuntimeWarning: divide by zero encountered in log10
  log_col_dens = np.log10(np.array(col_lines_split[1::2], dtype=float))


In [22]:
for s in list(species_logN_samples_final.keys()):
    if np.any(species_logN_samples_final[s].flatten()==-99):
        print('Error')

Save column density grid

In [23]:
output = open(rootdir+'final_grid_fixT_z126.pkl', 'wb')
pickle.dump(species_logN_samples_final, output)
output.close()